# Esempio RNN base con sequenza di Fibonacci

In questo esempio si genera una sequenza di fibonacci (una lista di numeri) lunga n, si divide in finestre di lunghezza fissa e si usa la RNN per prevedere il prossimo numero della sequenza a partire dalla finestra data.

 Generato da ProfAI - https://prof.profession.ai/

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, InputLayer
from keras.backend import clear_session

In [2]:
# Generare la sequenza di Fibonacci
def fibonacci_sequence(n):
    seq = [0, 1]
    for i in range(2, n):
        seq.append(seq[i-1] + seq[i-2])
    return seq

In [3]:
# Preparare i dati
sequence_length = 20
sequence = fibonacci_sequence(sequence_length)

X = []
y = []
window_size = 2

for i in range(len(sequence) - window_size):
    X.append(sequence[i:i+window_size]) #finestra della sequenza
    y.append(sequence[i+window_size]) #prossimo numero dopo finestra presa

X = np.asarray(X).astype(int)
y = np.asarray(y).astype(int)

In [4]:
print(X.shape)
print(y.shape)

(18, 2)
(18,)


In [5]:
X[-1]

array([1597, 2584])

In [6]:
y[-1]

4181

Arrivati qui, X è la matrice di input, che contiene le sequenze. 
Cioè ogni riga è una sequenza lunga window campioni.

Ad es. se X ha forma (95,5) significa che è composta da 95 sequenze (cioè 95 osservazioni/record) ognunga lunga 5 campioni. 

y è il vettore di ouput ed ha una sola dimensione, pari al numero di osservazioni in X (in modo classico). Qui però y non contiene la classe corrispondente alla sequenza, ma il suo valore numerico (qui il task è più di regressione che di classificazione)

In [7]:
# Rimodellare i dati per l'input dell'RNN
X = X.reshape((X.shape[0], X.shape[1], 1))
print(X.shape)

(18, 2, 1)


In [8]:
X[0]

array([[0],
       [1]])

Bisogna fare reshape per quanto si aspetta la RNN in input.
Dalla doc di keras:
> sequence: A 3D tensor, with shape [batch, timesteps, feature].

Quindi l'input deve avere per forza 3 dimensioni (altrimenti c'è errore al momento del fit/costruzione modello):
1. La prima dimensione si riferisce al batch (numero di osservazioni);
1. La seconda dimensione si riferisce ai campioni temporali (lunghezza sequenza);
1. La terza dimensione sono le features associate ad ogni record.

Nel caso più semplice (es. con segnali numerici, come in questo esempio), si ha una sola feature, che è il valore numerico a quell'istante temporale, quindi la terza dimensione vale 1.

NB: in questo caso particolare la terza dimensione è superflua di fatto, poichè posso mettere il valore del segnale già alla seconda dimensione (essendo tale valore uno scalare); però la RNN è utilizzabile anche con segnali "multidimensionali", es con il testo, dove ogni parola (che si può pensare come un campione temporale vero e proprio) viene mappata su N dimensioni, cioè ha N features e in quel caso il tensore prodotto da embedding avrà le tre dimensioni diverse da 1.

### Modello

In [9]:
clear_session()
# Creare il modello RNN
model = Sequential()
# NB: window_size vale 5
model.add(InputLayer(shape=(window_size,1)))
# model.add(SimpleRNN(10, input_shape=(window_size, 1)))
model.add(SimpleRNN(10, activation='tanh'))
model.add(Dense(1, activation='linear'))

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 10)             │           120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131 (524.00 B)

 Trainable params: 131 (524.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(optimizer='adam', 
              loss='mean_squared_error')

In [11]:
# Addestrare il modello
model.fit(X, y, epochs=100)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 877ms/step - loss: 1572567.3750
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1572553.8750
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1572540.5000
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1572526.7500
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1572513.7500
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1572500.1250
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1572486.7500
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1572473.0000
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1572459.8750
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1572446.2500
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1572432.8750
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1572419.3750
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1572405.8750
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 1572392.5000


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1571142.0000
Epoch 79/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1571128.7500
Epoch 80/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1571115.5000
Epoch 81/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1571101.8750
Epoch 82/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1571088.6250
Epoch 83/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1571075.1250
Epoch 84/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1571061.7500
Epoch 85/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1571048.3750
Epoch 86/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1571035.1250
Epoch 87/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1571021.6250
Epoch 88/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1571008.3750
Epoch 89/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1570994.7500
Epoch 90/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1570981.5000
Epoch 91/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1570968.0000
Epoch

In [12]:
# Prevedere il prossimo numero nella sequenza
test_input = np.array([sequence[-window_size:]]).astype(int)
test_input = test_input.reshape((1, window_size, 1))
predicted_number = model.predict(test_input, verbose=0)

print(f"Il prossimo numero previsto nella sequenza è: {predicted_number[0][0]}")



Il prossimo numero previsto nella sequenza è: 0.4345535635948181


In [13]:
test_input

array([[[2584],
        [4181]]])

In [14]:
2584+4181

6765

In [15]:
model.layers[0].get_weights()[0]

array([[ 0.39414224,  0.60170704,  0.01346342, -0.3305009 ,  0.4393254 ,
        -0.41807514,  0.40292194, -0.19272691,  0.40427488,  0.3645335 ]],
      dtype=float32)

In [16]:
len(model.layers)

2